In [5]:
import os

import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

from trackml.dataset import load_event
from trackml.score import score_event

#from utils.session import Session
#from geometric.helix import HelixUnroll
#from geometric.tools import merge_naive, reassign_noise, label_encode, hit_completeness
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cluster import dbscan
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder

In [6]:
def label_encode(y):
    return LabelEncoder().fit_transform(y)


def reassign_noise(labels: np.ndarray, idx):
    """
    assign noisy points (labeled with key_value such as -1 or 0) to their own clusters of size 1
    """
    ret = labels.copy()
    ret[idx] = np.arange(np.sum(idx)) + np.max(ret) + 1
    return ret


def merge_naive(pred_1, pred_2, cutoff=20):
    """
    naive cluster merging:
    iterate over hits; if a hit belongs to a larger cluster in pred_2, it is reassigned
    """
    if pred_1 is None:
        return pred_2
    c1, c2 = Counter(pred_1), Counter(pred_2)  # track id -> track size
    n1, n2 = np.vectorize(c1.__getitem__)(pred_1), np.vectorize(
        c2.__getitem__)(pred_2)  # hit id -> track size
    pred = pred_1.copy()
    idx = (n2 > n1) & (n2 < cutoff)
    pred[idx] = max(pred_1) + 1 + pred_2[idx]
    return label_encode(pred)


def merge_discreet(pred_1, pred_2, cutoff=21):
    """
    discreet cluster merging (less likely to reassign points)
    iterate over clusters in pred_2; np.sum(n1[idx]) < c2[track]**2 -> pred[idx] = d + track
    this is self-documenting
    """
    if pred_1 is None:
        return pred_2
    c1, c2 = Counter(pred_1), Counter(pred_2)  # track id -> track size
    n1, n2 = np.vectorize(c1.__getitem__)(pred_1), np.vectorize(
        c2.__getitem__)(pred_2)  # hit id -> track size
    pred = reassign_noise(pred_1, n1 > cutoff)
    pred_2 = reassign_noise(pred_2, n2 > cutoff)
    n1[n1 > cutoff] = 1
    n2[n2 > cutoff] = 1
    d = max(pred) + 1
    for track in c2:
        if c2[track] < 3:
            continue
        idx = pred_2 == track
        if np.sum(n1[idx]) < c2[track]**2:
            pred[idx] = d + track
    return label_encode(pred)


def hit_completeness(df, idx, track_size=None):
    """
    (the number of non-noisy hits in the idx) / (the total number of hits from all particles
    that have at least 1 hit in the idx)
    """
    if track_size is None:
        track_size = df.groupby("particle_id")["x"].agg("count")
    num = (df.loc[idx, "particle_id"] != 0).sum()
    all_particles = df.loc[idx, "particle_id"].unique().tolist()
    if 0 in all_particles:
        all_particles.remove(0)
    denom = track_size[all_particles].sum()
    return num / denom


def track_completeness(df, idx):
    """
    (number of tracks with all hits in the region) / (number of tracks that have at least 1 hit in the region)
    idx is a boolean mask over the region
    """
    all_particles = df.loc[idx, "particle_id"].unique().tolist()
    if 0 in all_particles:
        all_particles.remove(0)

    agg_1 = df.loc[idx, :].groupby("particle_id", sort=True)["x"].agg("count")
    if 0 in agg_1:
        agg_1.drop(0, inplace=True)
    agg_2 = df.loc[df.particle_id.isin(all_particles), :].groupby(
        "particle_id", sort=True)["x"].agg("count")
    return np.mean(agg_1 == agg_2)


In [7]:
class HelixUnroll(object):
    def __init__(
            self,
            # helix-unrolling parameters
            r3_func=lambda x, y, z: np.sqrt(x**2 + y**2 + z**2),
            dz_func=lambda i: (-1)**(i+1) * (-7e-4 + i * 1e-5),
            n_steps=150,
            hidden_transform=lambda x: x * np.array([1.0, 1.0, 0.75]),
            # cluster aggregation parameters
            merge_func=merge_naive,
            # dbscan parameters
            eps_func=lambda i: 3.5e-3 + 5e-6 * i,
            p=2,
            dbscan_n_jobs=-1
            ):
        self.r3_func = r3_func
        self.dz_func = dz_func
        self.n_steps = n_steps
        self.hidden_transform = hidden_transform  # transform the hidden space after scaling, before dbscan

        self.merge_func = merge_func

        self.eps_func = eps_func
        self.p = p
        self.dbscan_n_jobs = dbscan_n_jobs

    def fit_predict(self, df, score_func=None, verbose=False,i):
        df = df.copy()
        df.loc[:, "r3"] = self.r3_func(df.x, df.y, df.z)

        # df.loc[:, "rs"] = np.sqrt(df.x ** 2 + df.y ** 2 + df.z ** 2)  # radius in spherical coordinate system
        df.loc[:, "rt"] = np.sqrt(df.x ** 2 + df.y ** 2)  # radius in cylindrical coordinate system
        df.loc[:, "a0"] = np.arctan2(df.y, df.x)  # angle in cylindrical coordinate system
        df.loc[:, "z1"] = df.z / df.rt  # monotonic with cot(psi)
        # df.loc[:, "z2"] = df.z / df.rs TODO: 4 feature maybe? [1, 1, 0.4, 0.4]

        pred = []
        score_list = []

        
        dz = self.dz_func(i)
        df.loc[:, "a1"] = df.a0 + dz * df.r3  # rotation, points with higher r3 are rotated to a larger degree
        # convert angle to sin/cos -> more intuitive in Euclidean distance
        # e.g. 2pi and 0 should be very close
        df.loc[:, "sina1"] = np.sin(df.a1)
        df.loc[:, "cosa1"] = np.cos(df.a1)
        # scale the space
        dfs = scale(df.loc[:, ["sina1", "cosa1", "z1"]])
        # use hidden transformation methods to re-weight the features. Consider nonlinear transformations later.
        dfs = self.hidden_transform(dfs)

            """
            if score_func is not None:
                # use a callback to customize scoring
                step_score = score_func(pred)
                score_list.append(step_score)
                if verbose:
                    print(str(i).rjust(3) + ": {:.6f}".format(step_score))
            """
        return dfs

In [8]:
class Session(object):
    """
    A highly integrated framework for efficient data loading, prediction submission, etc. in TrackML Challenge
    (improved version of the official TrackML package)

    Precondition: the parent directory must be organized as follows:
    - train (directory)
        - event000001000-cells.csv
        ...
    - test (directory)
        - event000000001-cells.csv
        ...
    - detectors.csv
    - sample_submission.csv
    """
    # important constants to avoid spelling errors
    HITS = "hits"
    CELLS = "cells"
    PARTICLES = "particles"
    TRUTH = "truth"

    def __init__(self, parent_dir="./", train_dir="train/", test_dir="test/", detectors_dir="detectors.csv",
                 sample_submission_dir="sample_submission.csv"):
        """
        default input:
        Session("./", "train/", "test/", "detectors.csv", "sample_submission.csv")
        Session(parent_dir="./", train_dir="train/", test_dir="test/", detectors_dir="detectors.csv", sample_submission_dir="sample_submission.csv")
        """
        self._parent_dir = parent_dir
        self._train_dir = train_dir
        self._test_dir = test_dir
        self._detectors_dir = detectors_dir
        self._sample_submission_dir = sample_submission_dir

        if not os.path.isdir(self._parent_dir):
            raise ValueError("The input parent directory {} is invalid.".format(self._parent_dir))

        # there are 8850 events in the training dataset; some ids from 1000 to 9999 are skipped
        if os.path.isdir(self._parent_dir + self._train_dir):
            self._train_event_id_list = sorted(
                set(int(x[x.index("0"):x.index("-")]) for x in os.listdir(self._parent_dir + self._train_dir)))
        else:
            self._train_dir = None
            self._train_event_id_list = []

        if os.path.isdir(self._parent_dir + self._test_dir):
            self._test_event_id_list = sorted(
                set(int(x[x.index("0"):x.index("-")]) for x in os.listdir(self._parent_dir + self._test_dir)))
        else:
            self._test_dir = None
            self._test_event_id_list = []

        if not os.path.exists(self._parent_dir + self._detectors_dir):
            self._detectors_dir = None

        if not os.path.exists(self._parent_dir + self._sample_submission_dir):
            self._sample_submission_dir = None

    @staticmethod
    def get_event_name(event_id):
        return "event" + str(event_id).zfill(9)

    def get_train_events(self, n=10, content=(HITS, TRUTH), randomness=True):
        n = min(n, len(self._train_event_id_list))
        if randomness:
            event_ids = np.random.choice(self._train_event_id_list, size=n, replace=False).tolist()
        else:
            event_ids, = self._train_event_id_list[:n]
            self._train_event_id_list = self._train_event_id_list[n:] + self._train_event_id_list[:n]

        event_names = [Session.get_event_name(event_id) for event_id in event_ids]
        return event_names, \
            (load_event(self._parent_dir + self._train_dir + event_name, content) for event_name in event_names)

    def remove_train_events(self, n=10, content=(HITS, TRUTH), randomness=True):
        """
        get n events from self._train_event_id_list:
        if random, get n random events; otherwise, get the first n events
        :return:
         1. ids: event ids
         2. an iterator that loads a tuple of hits/cells/particles/truth files
        remove these train events from the current id list
        """
        n = min(n, len(self._train_event_id_list))
        if randomness:
            event_ids = np.random.choice(self._train_event_id_list, size=n, replace=False).tolist()
            for event_id in event_ids:
                self._train_event_id_list.remove(event_id)
        else:
            event_ids, self._train_event_id_list = self._train_event_id_list[:n], self._train_event_id_list[n:]

        event_names = [Session.get_event_name(event_id) for event_id in event_ids]
        return event_names, \
            (load_event(self._parent_dir + self._train_dir + event_name, content) for event_name in event_names)

    def remove_test_events(self, n=10, content=(HITS, CELLS), randomness=False):
        n = min(n, len(self._test_event_id_list))
        if randomness:
            event_ids = np.random.choice(self._test_event_id_list, size=n, replace=False).tolist()
            for event_id in event_ids:
                self._test_event_id_list.remove(event_id)
        else:
            event_ids, self._test_event_id_list = self._test_event_id_list[:n], self._test_event_id_list[n:]
        event_names = [Session.get_event_name(event_id) for event_id in event_ids]
        return event_names, \
            (load_event(self._parent_dir + self._test_dir + event_name, content) for event_name in event_names)

    def make_submission(self, predictor, path):
        """
        :param predictor: function, predictor(hits: pd.DataFrame, cells: pd.DataFrame)->np.array
         takes in hits and cells data frames, return a numpy 1d array of cluster ids
        :param path: file path for submission file
        """
        sub_list = []  # list of predictions by event
        for event_id in self._test_event_id_list:
            event_name = Session.get_event_name(event_id)

            hits, cells = load_event(self._parent_dir + self._test_dir + event_name, (Session.HITS, Session.CELLS))
            pred = predictor(hits, cells)  # predicted cluster labels
            sub = pd.DataFrame({"hit_id": hits.hit_id, "track_id": pred})
            sub.insert(0, "event_id", event_id)
            sub_list.append(sub)
        final_submission = pd.concat(sub_list)
        final_submission.to_csv(path, sep=",", header=True, index=False)

In [10]:
s1 = Session(parent_dir="/home/alexanderliao/data/GitHub/Kaggle-TrackML/portable-dataset/")

In [55]:
class HelixClassifier(BaseEstimator, ClassifierMixin):

     def __init__(self,helix,i,lo,hi):
         self.helix = helix
         self.i = i
         self.lo = lo
         self.hi = hi

     def fit(self, X, y):
         self.X_ = X
         self.y_ = y 
         # Return the classifier
         return self

     def predict(self, X):    
          X.loc[:, "psi"] = np.arctan2(np.sqrt(X.x ** 2 + X.y ** 2), np.abs(X.z))
          idx = (X.psi >= np.deg2rad(self.lo)) & (X.psi < np.deg2rad(self.hi))
          return dbscan(X= self.helix.fit_predict(X.loc[idx, :], self.i), eps=self.helix.eps_func(i), min_samples=1, metric="minkowski", p=self.helix.p, n_jobs=self.helix.dbscan_n_jobs)[1]
        
        

In [56]:
n_events = 1
preds=[];
classifiers=[];
clabels=[];
voters=[];

In [57]:
unroller = HelixUnroll(
        r3_func=lambda x, y, z: np.sqrt(x ** 2 + y ** 2 + z ** 2),
        dz_func=lambda i: (-1) ** (i + 1) * (-7e-4 + i * 1e-5),
        n_steps=120,
        hidden_transform=lambda x: x * np.array([1.0, 1.0, 0.75]),
        merge_func=merge_naive,
        eps_func=lambda i: 3.5e-3 + 5e-6 * i,
        p=2,
        dbscan_n_jobs=-1
    )

In [58]:
for i in range(unroller.n_steps):
    voter = HelixClassifier(unroller,i,0,20).fit([1],[2])
    classifiers.append(voter)
    clabels.append(str(i))
    voters.append((str(i),voter))

In [59]:
assembly = VotingClassifier(estimators=voters, voting='hard')
print(assembly)
assembly.fit([1],[2])

VotingClassifier(estimators=[('0', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=0, lo=0)), ('1', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=1, lo=0)), ('2', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, h...HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=119, lo=0))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)


VotingClassifier(estimators=[('0', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=0, lo=0)), ('1', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=1, lo=0)), ('2', HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, h...HelixClassifier(helix=<__main__.HelixUnroll object at 0x7fb17c9b68d0>, hi=20,
        i=119, lo=0))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [ ]:
results=[]
for hits, truth in s1.get_train_events(n=n_events, content=[s1.HITS, s1.TRUTH], randomness=True)[1]:
     print("=" * 120)
     hits = hits.merge(truth, how="left", on="hit_id")
     results.append(assembly.predict(hits))
    

In [ ]:
   for hits, truth in s1.get_train_events(n=n_events, content=[s1.HITS, s1.TRUTH], randomness=True)[1]:
        print("=" * 120)
        hits = hits.merge(truth, how="left", on="hit_id")
        preds.append(subroutine_psi_slice(hits, 5, 10))
        print(len(preds[0]))
    
    classifiers = []
    i = 0
    for pred in preds:
        for res in pred:
            voter = DummyClassifier(data=res).fit([1],[2])
            classifiers.append((str(i),voter))
            i = i + 1
        vote = VotingClassifier(estimators=classifiers, voting='hard')
        vote.fit([1,2],[2])
        class_res = vote.predict([1]*17650)